In [1]:
import numpy as np
from PIL import ImageGrab
import cv2
import time
from numpy import ones,vstack
from numpy.linalg import lstsq
from directkeys import PressKey, W, A, S, D
from statistics import mean
import os
from scipy import optimize
from matplotlib import pyplot as plt, cm, colors
from directkeys import PressKey,ReleaseKey, W, A, S, D
import random

In [2]:
def region_of_interest(image):#enclosed region of our field of view and recall that the enclosed region was triangular in shape
    # height = image.shape[0] #limit the extent of our field of view
    polygons = np.array([[
        [10,500],[10,300], [300,200], [500,200], [800,300], [800,500]]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [3]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:#lines=[[[2,3,2,3]]]
            # if line is not None and len(line) == 4:
            x1, y1, x2, y2 = line
            try:
                x1, y1, x2, y2 = x1.astype(int), y1.astype(int), x2.astype(int), y2.astype(int)
                
                #(image, start_point, end_point, color, thickness
                cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 30)
            except ValueError as e:
                print(f"Error converting line coordinates to integers: {e}")
    else:
        pass
       
    return line_image

In [4]:
def canny(image):
    gray=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0) #kernel, deviation
    canny=cv2.Canny(blur, 200, 200)
    return canny

In [5]:
def make_coordinates(image,line_parameters):
    slope,intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1,y1,x2,y2])

In [6]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    if lines is None:
        return np.array([])

    for line in lines:
        x1, y1, x2, y2 = line
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))

    #operate vertically along the rows to get the average slope and the average y-intercept respectively
    left_line = make_coordinates(image, np.average(left_fit, axis=0)) if left_fit else None
    #average slope, y-intercept of a single line on right side
    right_line = make_coordinates(image, np.average(right_fit, axis=0)) if right_fit else None

    lines_to_return = []
    if left_line is not None:
        lines_to_return.append(left_line)
    if right_line is not None:
        lines_to_return.append(right_line)
    
    return np.array(lines_to_return)

In [11]:
bbox=(0, 40, 800, 640)
def main():
    while True:
        screen = np.array(ImageGrab.grab(bbox=bbox)) #left, top, right, bottom
        frame = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
        
        canny_image = canny(frame)
        cropped_image = region_of_interest(canny_image)

        #(img, rho, theta, threshold, minLineLength,maxLineGap)
        lines = cv2.HoughLinesP(cropped_image, 2, np.pi / 180, 100, minLineLength=40, maxLineGap=5)
        if lines is not None:
            lines = [line[0] for line in lines]  # Flatten the lines array
        averaged_lines = average_slope_intercept(frame, lines)
        line_image = display_lines(frame, averaged_lines)
        combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
        cv2.imshow("result", combo_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

In [13]:
main()